In [ ]:
# In this file, we verify Lemma 3.6, Lemma 3.9 for inductions
# See Appendix B for meaning of some variables and instructions. 
# Press run all to run the code and verify the estimates 

# p, d parameters,  l = 4 / (p-1) + 1, ga short for gamma 
# Num, number of comptuered series

l, ga = var('l ga')

d = 4
Num = 500
 
# Parameters in Section 2.1 
ep = l * ga**2 - 1
A = d + 1 - (d - 1 - 2 * l) * ga
B = 2 * d - 1 - l 

# (U, Y) ODE system defined in Section 2.2 
# fy(Y) is the same as f(Y) in the paper, and it is not the derivative
fy(Y) = - ep - A * Y + B * Y**2
del_U(Y, U) = 2 * U * ( U + fy(Y) + (d-1) * Y * (1 - Y) )
del_Y(Y, U) = ( - 1 + d * Y ) * U + ( Y-1) * fy(Y) 


# degU = dU, degY = dY in Section 2.2 in the paper, which are degree of del_a as polynomials in U, a = Y, U.
# The following output is symbolic, we need to manully change to integer
degU = 2
degY = 1

# (Y0, U0) sonic point at the (Y, U) system 
Y0 = 0
U0 = ep 

# The gradient of del_U, del_Y at the spnic point defined in Section 2.4 
c1 = del_U.diff(U)(Y = Y0, U = U0).full_simplify()
c3 = del_U.diff(Y)(Y = Y0, U = U0).full_simplify()

c2 = del_Y.diff(U)(Y = Y0, U = U0).full_simplify()
c4 = del_Y.diff(Y)(Y = Y0, U = U0).full_simplify()

# U1 = d U / d Y at the sonic point, lam_neg = \lambda_-, and U1 are defined in Section 2.4 
lam_neg = ( (c1 + c4) - ( (c1 - c4)**2 + 4 * c2 * c3 ) **(1/2) ) / 2
U1 =  ( lam_neg - c4 ) / c2 

# Create a power series placeholder for U, range include 0,1.., Num;
x = var("x")
UU = list(var('U_%d' % i) for i in (0..Num))

# PP, save the polynomials in the coefficients 
FF = list(var('F_%d' % i) for i in (0..2))
GG = list(var('G_%d' % i) for i in (0..2))

# Define Basic operation for power series expansion 
# Power_prod get power series of A * B up to n terms, following Section 3.1 
# len(A) - 1, len(B) - 1 are the upper bound of degree of polynomials A, B
def Power_prod(n, A, B):
    C = list(var('C_%d' % i) for i in (0..n))
    for i in (0..n):
        C[i] = sum( A[i-j] * B[j] for j in (max(i+1-len(A), 0)..min(len(B)-1,i)  ) )
    return C 

# return power series coefficients of c1 * A + c2 * B up to n terms 
def Power_sum(n, A, B, c1, c2):
    C = list(var('C_%d' % i) for i in (0..n))
    for i in (0..n):
        C[i] = 0
        if i <= len(A)-1: 
            C[i] = C[i] + c1 * A[i]    
        if i <= len(B)-1: 
            C[i] = C[i] + B[i] * c2
            
    return C 

# First compute the Power series expansion of del_U, del_Y in U. See Section 2.2 
# del_U = sum_i F_i(Y) * U^i, del_Y = sum_i G_i(Y) * U^i. FF[i] denote F_i, GG[i] denote G_i
# d_U (del_U) = sum_i F_i(Y) * i * U^{i-1},  d_U(del_Y) = sum_i G_i(Y) * i * U^{i-1}

# Then compute the power series coefficients DelUn, DelYn of del_U(Y, U(Y)), del_Y(Y, U(Y)) in Y
# dU_DelUn, dU_DelYn are n-th ppower series coefficients of (d_U(del_U)) (Y, U(Y)) , (d_U( del_Y))( Y, U(Y))
# Input n, we only compute the first n power series coefficients 
def Power_Del(n):
    # Given power series of U : UU[0], UU[1],... as symbolic value
    # U_pow[i] store power series coefficients for U^i. U^0 = 1. 
    U_pow = [ [1], UU]
    deg = max(degU, degY)
    U_now = UU 
    for i in (2.. deg) :
        U_next = Power_prod(n, U_now, UU)
        U_pow.append( U_next) 
        U_now = U_next 
    
    # Get the coeffiicents as polynomial in xi = Y -  Y0 
    xi = var('xi')

    for i in (0..deg):
        if i <= degU:                      
            # Derive the coefficients of U^i in del_U, and then expand the Y polynoamils as xi, for Y = Y0 + xi
            ff0(Y) = ( (del_U(Y, U).diff(U, i) / factorial(i))(U =0) ).full_simplify()    
            FF[i] = ff0(Y = xi + Y0).list(xi)   
    
        if i <= degY:
            # Do the same for del_Y 
            gg0(Y) = ( (del_Y(Y, U).diff(U, i) / factorial(i))(U =0) ).full_simplify()  
            GG[i] = gg0(Y = xi+ Y0).list(xi)

    for i in (0..deg):  
        # Assemble the power series for del_U=  sum_i F_i(Y) * U^i, and  del_Y, d_U (del_U),  d_U(del_Y) 
        # Initialize with the first summand
        if i == 0 :
            # vanishes at the sonic point 
            DelUn = FF[0]
            DelYn = GG[0]
                    
            dU_DelUn = FF[1]
            dU_DelYn = GG[1]
        else: 
            # add the expansion from F_i(Y) * U^i to DelUn, and G_i(Y) * U^i to DelYn
            if i <= degU :        
                DelUn = Power_sum(n, DelUn, Power_prod(n, FF[i], U_pow[i] ), 1, 1 )

            if i<= degY :
                DelYn = Power_sum(n, DelYn, Power_prod(n, GG[i], U_pow[i] ), 1, 1 )

            # Below, assemble expression for
            #  d_U (del_U) = sum_i F_i(Y) * i * U^{i-1},  d_U(del_Y) = sum_i G_i(Y) * i * U^{i-1}
            # Add expansion from F_{i+1}(Y) * (i+1) * U^i, and G_{i+1} * (i+1) * U^i
            if i <= degU - 1:    
                FFm = list( FF[i+1][j] * (i+1) for j in (0.. (len(FF[i+1]) - 1 ) ) )
                dU_DelUn = Power_sum(n, dU_DelUn, Power_prod(n, FFm, U_pow[i] ), 1, 1 )
            
            if i <= degY - 1:
                GGm = list( GG[i+1][j] * (i+1) for j in (0.. (len(GG[i+1]) - 1 ) ) )
                dU_DelYn = Power_sum(n, dU_DelYn, Power_prod(n, GGm, U_pow[i] ), 1, 1 )
        
    return DelUn, DelYn, dU_DelUn, dU_DelYn, FF, GG  

DelUn, DelYn, dU_DelUn, dU_DelYn , FF, GG   = Power_Del(Num)

In [ ]:

# Set the precision for the interval arithmetic 
prec = 200 

# We use X_V to denote the interval arithmetic value for X. X_R for floating point value 
# X_R default value type for X 
RR = RealBallField(prec)
l_R = 5/3 
l_V = RR(l_R)

# coe(i) for the coefficient of U_i in the recursive formula in Lemma 3.2
# Two eigenvalues: 0 < lam_s < lam_l,  s for small, l for large
coe(i) = i * (c2 * U1 + c4) + c2 * U1 - c1 
lam_s = c2 * U1 + c4
lam_l = c1 - c2 * U1 

# DelUn, DelYn are coefficients of the power series expansion of del_U, del_Y
# Compute the power series coefficients of U following Lemma 3.2 
# DUn, DYn stores these values. 
def solu(ga_V, I_show):
    solu = {ga : ga_V , l:l_V}

    # First assemble the first two coefficients for U_i, Delta_{U, i}, Delta_{Y,i}, i=0,1
    DUn = [RR(0)]
    DYn = [RR(0)]

    U1_V = RR( U1(solu))

    solu[UU[0]] = RR( U0(solu))
    solu[UU[1]] = U1_V 
    DUn.append(RR(DelUn[1]( solu)))
    DYn.append(RR(DelYn[1]( solu)))

    if I_show == 1:
        print("U0: ", solu[UU[0]])
        print("U1: ", solu[UU[1]])

    solu1 = solu 

    c1_V = c1(solu)
    c2_V = c2(solu)

    for i in (2.. Num):

        # In solu1, at step i, we have values for U_0, .., U_{i-1}, but U_i = 0.
        # We use this to evaluate the right hand side of the recursive formula in Lemma 3.2
        # DUn0, DYn0 evaluate Delta_{U,i}, Delta_{Y,i} at (U_0, U_1,..,U_{i-1}, U_i = 0).
        solu1[UU[i]] = 0
        
        DUn0 = RR(DelUn[i](solu1))
        DYn0 = RR(DelYn[i](solu1))

        # Evaluating DYn[j], 2 \leq j \leq i-1, we do not need value of U[i].
        M = RR(sum( (i+1-j) * DYn[j] *  solu[UU[i+1-j]] for j in (2..i-1) ))
        
        solu[UU[i]] = RR( (DUn0 - U1_V * DYn0 - M ) / ( RR(coe(i)(solu) ) ) )
        # With UU[i], update the value of DUn, DYn for (del_U)_i, (del_Y)_i using formulas in Lemma 3.2      
        DUn.append( DUn0 + c1_V *  solu[UU[i]])
        DYn.append( DYn0 + c2_V *  solu[UU[i]])

        if I_show == 1: 
            # Show that U_i / U_{i-1} grow linearly. 
            print("U_i / U_{i-1} - U_{i-1}/U_{i-2}:" + str(i) +": ", \
            solu[UU[i]] / solu[UU[i-1]] -  solu[UU[i-1]] /solu[UU[i-2]] )

        # solu1 stores computed value of U[0],.,U[i]
        solu1 = solu 

    return solu, DUn, DYn 

solu0 = solu(RR(l_V**(-1/2)) , 1)[0]


U0:  [+/- 5.21e-60]
U1:  [5.2581988897471611256786176933188266407221947803527727217725 +/- 9.20e-59]
U_i / U_{i-1} - U_{i-1}/U_{i-2}:2:  nan
U_i / U_{i-1} - U_{i-1}/U_{i-2}:3:  [2.44654671382004691885477796148869894207186634763943057175 +/- 8.34e-57]
U_i / U_{i-1} - U_{i-1}/U_{i-2}:4:  [1.1882185710634274637451122668865391353219380445515906007 +/- 2.97e-56]
U_i / U_{i-1} - U_{i-1}/U_{i-2}:5:  [0.890928739601624679451331381583701862009349769238806570 +/- 3.14e-55]
U_i / U_{i-1} - U_{i-1}/U_{i-2}:6:  [0.772641209252783293678664739421655054499358685424265461 +/- 8.19e-55]
U_i / U_{i-1} - U_{i-1}/U_{i-2}:7:  [0.74646179868211290280570195921087108232219987468159340 +/- 3.04e-54]
U_i / U_{i-1} - U_{i-1}/U_{i-2}:8:  [0.78172674952528941789685810110864448484200009416823913 +/- 6.00e-54]
U_i / U_{i-1} - U_{i-1}/U_{i-2}:9:  [0.8610637258952767226110322068323066452325619491174394 +/- 2.31e-53]
U_i / U_{i-1} - U_{i-1}/U_{i-2}:10:  [0.9676455803351468529316121295385886973627917813396595 +/- 4.94e-5

In [ ]:
# Define the constants and parameters in the induction following Section 3.2
n0 = 20

N = 30
n1 = 450 
del_hat = RR('0.049')
C1_L = []

dU = 2
dY = 1

lenF = list( len(FF[i]) for i in (0..dU)) 
lenG = list( len(GG[i]) for i in (0..dY))

dF = max(lenF)-1
dG = max(lenG)-1

# C_mfr is the Catalan number.  U_hat is the renormalized power series coefficients. 
# Both are defined at the end of Section 3.1. 
C_mfr = [  1 / (i+1) * factorial(2 * i) / ( factorial(i) )^2 for i in (0..Num) ]
U_hat = [ solu0[UU[i]] / C_mfr[i] for i in (0..Num )]

# First work out constant C1 in Inequality (a) in Lemma 3.5 for n <= n_0. 
# This is not a rigorous proof step. We will verify it for n <= n_1 below. 
# Lemma 3.5 Item (a) holds trivially for i = 0 with C1 = a 
for i in (1..n0):
    a = max(1, abs( U_hat[0] ) )
    for j in (0..i):
        a = max( a, abs( U_hat[j])  * abs( U_hat[i- j] ) / ( abs( U_hat[i] )  )  )
    C1_L.append(a)
    
C1 = max(C1_L)
C1 = max(RR('12.46'), C1)

# C_star defined in Inequality (d),(e) in Lemma 3.5
lam_l = c1 - c2 * U1  
C_star = DelYn[2](solu0) / lam_l(solu0)

l0 = 2

# Define the constants b_{i, j}, qn at the end of Section 3.2 
# b2_neg1 = b2_{2, -1} in the paper
b2 = [ C1 ** i * max( abs( U_hat[1]) , 1 )  for i in (0..l0)]
b2_neg1 = 1  

# b3 bound |U_hat[i] / U_hat[i+j]|. If j = 0, ratio = 1.
b3 = 1
for j in (1..l0):
    for i in (0.. N): 
        b3 = max(b3,  abs(U_hat[i] / U_hat[i+j]) )

qn1 = abs((n1 - N) * (1 - del_hat) * C_star ) / 4


# Define the functions e_i in Section 3.1.1 above Lemma 3.2 
# Power series coefficients c_{Y, j} for dU_DelY = -1 + d Y only has first two terms in its expansion in Y
e = list(  sum(  dU_DelYn[j](solu0) * (i+1-j) * solu0[UU[i+1-j]] for j in (0..min(i, 1)) ) \
    - dU_DelUn[i](solu0) for i in (0.. N +1))

# Define nu_l in Section 3.5.1 in the estimate of J1 
nu_l = vector(RR, n0+1)

for i in (1..l0):  
    # b_im2 for b[i-2]
    if i > 1:        
        b2_im2 = b2[i-2]
    elif i == 1: 
        # In this case, b2_im2 = b_{2, -1} = 1
        b2_im2 = 1 
    
    ll = list( abs( U_hat[m+2] ) * min( C1**(i-1) * qn1**(-m), b2_im2 * 2**(- (N-2-m) ) ) for m in (0..N-2))            
    UU02 = list( max(abs(  U_hat[i] ),1)   for i in (0..2) )
    nu_l[i] =  b3 * max(ll) +  max( max( abs(U_hat[0]), 1) * 2**(dG -1), max( UU02 ) * 2) * 2**(-N)   


# Bound Ji below following Section 3.5.1,  3.5.2.
# J1 <= C_J1 * n * |U_n-1|, J2 <= C_J2 * (n * |U_n-1|), J3 <= C_J3 * (n * |U_n-1|), err <= C_err * n * |U_n-1|
# b2_im2[0] is not used 
b2_im2 = vector(RR, l0 + 1)
b2_im2[2:l0] = b2[0:l0-2]
b2_im2[1] = 1 

# degree of G_j <= lenG[j] - 1
C_J1 = 4**(dY+2) * sum( sum( abs(GG[l][j](solu0)) * nu_l[l] for j in (0.. lenG[l] -1) ) for l in (1..dY) ) 
C_J2 = 4**(dY) * 2**(-N) * abs( solu0[UU[1]] ) \
    * sum( sum( abs(GG[l][j](solu0)) * b2_im2[l] * 2**(j+1)  for j in (0.. lenG[l] -1) )  for l in (1..dY)  )
C_J3 = 4 **(dY) * 2**(-N) \
    * sum( sum( abs(FF[l][j](solu0)) * b2_im2[l] * 2**(j+1)  for j in (0.. lenF[l] -1) )  for l in (1..dY)  )

# Bound in error following Section 3.5.3.  error <= tot_rel * (n * |U[n-1]|)
b41 = sum( abs(e[j]) * (3 * qn1)**(-( j-1 )) for j in (2..N-2) )
b42 = sum( abs(DelYn[j+1](solu0) ) * (3 * qn1)**(-( j-1 )) for j in (2..N-2) )
C_err = b42 + b41 / n1 

# Verify Lemma 3.9 in Section 3.5.4,  
# rel for Ineq (a), tot_rel for Ineq (c), qn1 for Ineq (b) in Lemma 3.9
rel =  C_err + C_J1 + (C_J2 + C_J3 + abs(e[1])  + abs(DelYn[2](solu0))  ) / n1 
tot_rel =  C_star / 4 * del_hat -  ( RR('0.05')  / lam_l(solu0) +  C_star * 3 / 2 / (4 * n1 - 2) ) 

if rel < del_hat and tot_rel > 0 and qn1 >  max( U_hat[1], 1):
    print('True: Verify Lemma 3.9')
else:
    print('False: error')



True: Verify Lemma 3.9


In [ ]:
# Check iterm (a) in Lemma 3.6, i.e. verify Lemma 3.4 for n <= n1
# Lemma 3.5 Item (a) holds trivially for i = 0 with this C1_n1
C1_n1 = max(1, abs( U_hat[0] ) )
for i in (1..n1):    
    for j in (0..i):
        C1_n1 = max( C1_n1, abs( U_hat[j])  * abs( U_hat[i- j] ) / ( abs( U_hat[i] )  ))


U_sign = [sign(U_hat[i]) for i in ( (n0-1)..n1)]
U_incre = [ U_hat[i] < U_hat[i+1] for i in ((n0-1)..n1-1)]

# U_rat check the growth rate in the asymptotics for \kappa = infty
U_rat = [  abs(U_hat[i] / U_hat[i-1] / i / C_star * 4 - 1) for i in ( (n0-1)..n1) ]
U_rat_max = max(U_rat)

if C1_n1 > C1 or (0 in U_sign) or (False in U_incre) or U_rat_max > del_hat:
    print('False: error')
else:
    print('True: Lemma 3.6 Item (a) is verified')



True: Lemma 3.6 Item (a) is verified


In [ ]:
# CHeck Item (b),(c)) in Lemma 3.6 
j0 = 25
rat = 0 

# Check inequality in item (b)
for i in (0..l0):
    for j in (2.. j0):
        # When j = 1,  rat_rhs = 1
        rat_rhs = abs(  U_hat[j+ i -1] ) /  max( abs(  U_hat[i] ) , 1 ) * 2**(j-1) 
        rat_rhs = max(rat_rhs, 1)
        rat_lhs = (1 - del_hat)**(j-1)  * (C_star / 4) ** (j-1) * (n1 + 1 - N - j0)**(j-1)
        rat = max( rat_rhs/rat_lhs, rat)

# Check inequality in item (c)
M1 = max( [ abs(U_hat[n0] ) / factorial(n0) / max( abs( U_hat[l] ), 1) * (C_star/ 4)**(l-n0) for l in (0..l0) ])
rat2 = ( 1 / (j0 + l0) ) ** l0 / M1 * (9/5) ** j0 / (3 * j0**(1/2))

# Want rat < 1, rat2 > 1   
if rat < 1 and rat2 > 1:
    print('True: Lemma 3.6 Item (b), (c) are verified')
    print([rat, rat2])
else:    
    print('False: error')


True: Lemma 3.6 Item (b), (c) are verified
[[0.0978897122792533746984490669040738017181581481488363972157 +/- 5.79e-59], [5.13207221901298968108355154005378195314277122663715 +/- 6.14e-51]]
